Support class to search where atomic positions start.

In [104]:
class SkipLines:
  def __init__(self, filename):
    self._skipheader = True
    self._filename = filename
    self.skip_lines_header = []
    self._generate_skip_bool_vector()
    self.skip_lines_header = [x for x, y in enumerate(self.skip_lines_header) if y]


  def _skip_header(self, line):
    if line.find('ATOMIC_POSITION') > -1:
      self._skipheader = False
    return self._skipheader


  def _generate_skip_bool_vector(self):
    with open(self._filename) as file:
      for line in file:
        self.skip_lines_header.append(self._skip_header(line))


Generates a column data with the names of elements in the same order described in input file. Here it takes in account that input file have a CELL_PARAMETERS card (skipfooter=4).

In [133]:
import pandas as pd
filename = 'hap_slab_sem_coment.cpmd.in'
sl = SkipLines(filename)

data = pd.read_csv(filename, sep=' ',
                   skipfooter=4, skiprows=sl.skip_lines_header,
                   names=['Element'], usecols=[0])

data.drop([0], inplace=True)

nat = len(data)

data.head()

<ipython-input-133-88c9487f1d42>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(filename, sep=' ',


,Element
1,Ca
2,Ca
3,Ca
4,Ca
5,Ca


In [134]:
elements_list = list(data.iloc[:, 0])

Updates the trajectory file with the correct elements names as the input file order.

In [139]:
traj_file = 'trajetoria.xyz'

new_file_list = []

with open(traj_file) as file:
  file_line_count = 0
  elements_count = 0
  for line in file:
    if file_line_count % (nat + 2) in [0, 1]:
      new_file_list.append(line)
      elements_count = 0
    else:
      element, v1, v2, v3, v4, v5, v6 = line.split()
      new_line = f'{elements_list[elements_count]: >2}   {v1:>15}{v2:>15}{v3:>15} {v4:>12}{v5:>12}{v6:>12}\n'
      new_file_list.append(new_line)
      elements_count += 1

    file_line_count += 1

new_file = ''.join(new_file_list)


Write the new file with elements names in correct order.

In [140]:
with open('trajetoria_novo.xyz','w') as file:
  file.write(new_file)